In [1]:
import os
import pickle

import numpy as np
import torch
from tqdm.auto import tqdm

from policy import config
from policy.dataset.dataset_loaders import dataset_loader
from policy.checkpoints import CheckpointIO

model_dir = "/home/mrl/Documents/Projects/tskill/out/VAE/009"
cfg_path = os.path.join(model_dir, "config.yaml")
cfg = config.load_config(cfg_path, None)

# Dataset
cfg["data"]["pad"] = False
cfg["data"]["augment"] = False
cfg["data"]["full_seq"] = False
cfg["data"]["max_count"] = 100
cfg["data"]["val_split"] = 0

# Load only the full episode version of the dataset
train_dataset, val_dataset = dataset_loader(cfg, return_datasets=True, 
                                            save_override=True,
                                            preshuffle=False,
                                            fullseq_override=True,
                                            )
print(len(train_dataset), len(val_dataset))
# Model
model = config.get_model(cfg, device="cuda")
checkpoint_io = CheckpointIO(model_dir, model=model)
load_dict = checkpoint_io.load("model_best.pt")
stt_encoder = model.stt_encoder
model.eval()

/home/mrl/miniconda3/envs/tskill/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Updating new train & val indices


/home/mrl/miniconda3/envs/tskill/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mrl/miniconda3/envs/tskill/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Recomputing scaling functions...
Computing seperate gripper scaling
Computing normal quantile transform
Computing linear scaling
Adding batch dimension to returned data!
50 0
freezing state encoder network!
/home/mrl/Documents/Projects/tskill/out/VAE/009/model_best.pt
=> Loading checkpoint from local file...
load state dict: <All keys matched successfully>


TSkillCVAE(
  (decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (linear1): Linear(in_features=1024, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=1024, bias=True)
        (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (norm3): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
        (dropout3): Dropout(p=0.1, inplace=False)
      )
    )

In [2]:
import h5py

# for dataset in val_dataset.sequence_datasets:
for dataset in [val_dataset]:
    val_dataset.data.close()

j = 0
# for dataset in train_dataset.sequence_datasets:
for dataset in [train_dataset]:
    print(f"##################### {j}")
    dataset_path = dataset.dataset_file
    print(f"==>> dataset_file: {dataset_path}")
    dataset.data.close()
    dataset_file = h5py.File(dataset_path, "r")
    dataset_file.close()
    dataset_file = h5py.File(dataset_path, "r+")
    dataset.data = dataset_file
    dataset.episodes = dataset_file["data"]
    for i in range(len(dataset)):
        eps = dataset.owned_indices[i]
        print(f"==>> eps: {eps}")
        trajectory = dataset.episodes[f"demo_{eps}"]
        print(f"==>> trajectory: {trajectory}")
        print(f"==>> trajectory: {trajectory.keys()}")
        if "resnet18" in trajectory["obs"].keys():
            if "img_feat" in trajectory["obs"]["resnet18"].keys():
                add_feats = False
            else:
                add_feats = True
            if "img_pe_1024" in trajectory["obs"]["resnet18"].keys():
                add_pe = False
            else:
                add_pe = True

            if not add_feats and not add_pe:
                continue
            else:
                print("MISSING ITEMS!!!!")
        
        rgb = dataset[i]["rgb"].to(model._device)
        print(f"==>> rgb.shape: {rgb.shape}")
        with torch.no_grad():
            img_feat, img_pe = stt_encoder(rgb)
        img_feat, img_pe = img_feat[:,0,...].detach().cpu().numpy(), img_pe[:,0,...].detach().cpu().numpy()
        print(f"==>> img_feat.shape: {img_feat.shape}")
        print(f"==>> img_pe.shape: {img_pe.shape}")
        # raise ValueError
        # # del trajectory["obs"]["image"]
        if add_feats:
            trajectory.create_dataset("obs/resnet18/img_feat",
                                    data=img_feat,
                                    dtype=img_feat.dtype,
                                    compression="gzip",
                                    compression_opts=5,)
        if add_pe:
            trajectory.create_dataset("obs/resnet18/img_pe_1024",
                                    data=img_pe,
                                    dtype=img_pe.dtype,
                                    compression="gzip",
                                    compression_opts=5,)
        
    j += 1

    dataset_file.close()


##################### 0
==>> dataset_file: /home/mrl/Documents/Projects/tskill/LIBERO/libero/datasets/libero_90/KITCHEN_SCENE1_open_the_top_drawer_of_the_cabinet_and_put_the_bowl_in_it_demo.hdf5
==>> eps: 0
==>> trajectory: <HDF5 group "/data/demo_0" (6 members)>
==>> trajectory: <KeysViewHDF5 ['actions', 'dones', 'obs', 'rewards', 'robot_states', 'states']>
MISSING ITEMS!!!!
==>> rgb.shape: torch.Size([1, 189, 2, 3, 128, 128])
==>> img_feat.shape: (189, 2, 16, 512)
==>> img_pe.shape: (189, 2, 16, 1024)
==>> eps: 1
==>> trajectory: <HDF5 group "/data/demo_1" (6 members)>
==>> trajectory: <KeysViewHDF5 ['actions', 'dones', 'obs', 'rewards', 'robot_states', 'states']>
MISSING ITEMS!!!!
==>> rgb.shape: torch.Size([1, 184, 2, 3, 128, 128])
==>> img_feat.shape: (184, 2, 16, 512)
==>> img_pe.shape: (184, 2, 16, 1024)
==>> eps: 2
==>> trajectory: <HDF5 group "/data/demo_2" (6 members)>
==>> trajectory: <KeysViewHDF5 ['actions', 'dones', 'obs', 'rewards', 'robot_states', 'states']>
MISSING ITE